# PyDSout: DataStream requests made simple

**Author:** Marcus Bravidor ([marcus.bravidor@hhu.de](mailto:marcus.bravidor@hhu.de))  
**Version:** PyDSout 0.1  
**Date:** 2018-04-04  
**License:** [MIT License](https://opensource.org/licenses/MIT) 

**Purpose:** Automatically request firm-level financial data (e.g., WorldScope) from Thomson Reuters DataStream and save the results to a long format CSV file (separator: semicolon or ";").

**A note on the example below:** Request for 20 items on 5 German companies.

## Setup

Loading PyDataStream and Login. Output of all accessable databases. _Datastream_ should be one of them.  
Replace `username`and `password` with the credentials supplied by your institution or company.

In [1]:
from pydatastream import Datastream
from suds.client import Client
DWE = Datastream(username="USERNAME", password="PASSWORD")
DWE.system_info()
DWE.sources()

['TMT',
 'Resolution',
 'Symbology',
 'XREF',
 'Economics',
 'ILXAUTH',
 'Datastream',
 'ILXMLP']

## Requesting time series data

### General

The request has a standardized structure: _Ticker_*~*_Item1, Item2, ItemN_*~*_EndDate_*~*_StartDate_*~*_Frequency_

Legend:
* *Ticker:* Ticker symbol from DataStream including possible special characters (@, :, etc.)
* *Item 1-Item N:* Item mnemonic from DataStream (e.g., WC...)
* *EndDate:* Date for last request in ISO format (YYYY-MM-DD)
* *StartDate:* Date for first request in ISO format (YYYY-MM-DD)
* *Frequency:* Number of points in time. Possible: Yearly (Y), Monthly (M), Weekly (W), Daily (D)

### Customization

Use the following steps to start a customized request:

* Change `startdate`, `enddate`, and `freq` to the desired values.
* Firms to query: Add a file named `firms.txt` to the `root` directory. Format:

```
DSID1
DSID2
DSID3

756944
866922
278419
```

* Add a file name `items.txt` to the `root` directory. Format:

```
code1,code2,code3,
NOSHFF,WC05301,WC07101,
```

### Limitations

**15 item limit**  
One request contains up to 15 items for all points in time (e.g., years) per firm. If you supply more than 15 items, the script automatically generates multiple requests per firm and merges the data in a later stage.  
Why the 15 items limit? From my tests, DataStream does not deliver more than that in one request. But feel free to try other values by changing `15` in line `itemslices = list(iterutils.chunked_iter(items[0], 15))`.  

**Multiple Requests at once**  
PyDataStream offers the option to [perform multiple requests at once](https://github.com/vfilimonov/pydatastream#debugging-and-error-handling) which would most likely make the script much faster. However, I did not implement this step, yet.

In [2]:
startdate = '2005-12-31'
enddate = '2016-12-31'
freq = 'Y'

import codecs, csv
with codecs.open("firms.txt", "rb", "utf-8") as file:
    f = csv.reader(file, delimiter="\n")
    firms = list(f)
    numfirms = len(firms)

with codecs.open("items.txt", "rb", "utf-8") as file:
    items = csv.reader(file, delimiter=",")
    items = list(items)

    i = 0
    nr = list()
    d = {}
    from boltons import iterutils
    itemslices = list(iterutils.chunked_iter(items[0], 15))
    numslices = len(itemslices)
    print ("Debugging information")
    print ("")
    for x in firms:
        j = 0
        d["raw{0}".format(i)] = {}
        print ("firm "+str(i+1)+" of "+str(numfirms)+" started.")
        for y in itemslices:
            types = ','.join(y)
            d["raw{0}".format(i)]["raw{0}".format(j)] = DWE.request(str(x[0])+"~="+str(types)+"~"+str(enddate)+"~"+str(startdate)+"~"+str(freq))
            j = j+1
        print ("firm "+str(i+1)+" of "+str(numfirms)+" finished.")
        i = i+1
print ("All requests completed.")

Debugging information

firm 1 of 5 started.
firm 1 of 5 finished.
firm 2 of 5 started.
firm 2 of 5 finished.
firm 3 of 5 started.
firm 3 of 5 finished.
firm 4 of 5 started.
firm 4 of 5 finished.
firm 5 of 5 started.
firm 5 of 5 finished.
All requests completed.


## Convert the unstructured query response to a structured list

**First Step: Convert the unstructured `suds` response to `JSON` format.** Code is unchanged from the suggestion by _radtek_ on [StackOverflow](https://stackoverflow.com/questions/17581731/parsing-suds-soap-complex-data-type-into-python-dict/17582946).

**Second Step: Convert the `JSON` data to a three-dimensional list.** The resulting list is structured as follows:

* First level: One entry per firm.
* Second level (rows in table below): One entry per item.
* Third level (columns in table below): One entry for (each item) per year. The only notable exemptions are static items (e.g., `DISPNAME`) with only one corresponding value (column 0).

Per firm, the strucutre looks as follows:

| item/value | 0 | 1 | 2 | ... | n |
|:---:|---:|---:|---:|:---:|---:|
| 0 | CCY | '' | | ... | |
| 1 | DATE | 2010-12-31T00:00:00 | 2011-12-31T00:00:00 | ... | 2016-12-31T00:00:00 |
| 2 | DISPNAME | CJ VOGEL FUR BETEILIG. DEAD - 26/03/14 | | ... | |
| 3 | WC05301 | 693.0 | 693.0 | ... | nan |
| ... | ... | ... | ...| ... | ... |
| n | ... | ... | ...| ... | ... |

The list containing all firms and datapoints is called `dobject`.

In [3]:
import json

def basic_sobject_to_dict(obj):
    """Converts suds object to dict very quickly.
    Does not serialize date time or normalize key case.
    :param obj: suds object
    :return: dict object
    """
    if not hasattr(obj, '__keylist__'):
        return obj
    data = {}
    fields = obj.__keylist__
    for field in fields:
        val = getattr(obj, field)
        if isinstance(val, list):
            data[field] = []
            for item in val:
                data[field].append(basic_sobject_to_dict(item))
        else:
            data[field] = basic_sobject_to_dict(val)
    return data


def sobject_to_dict(obj, key_to_lower=False, json_serialize=False):
    """
    Converts a suds object to a dict.
    :param json_serialize: If set, changes date and time types to iso string.
    :param key_to_lower: If set, changes index key name to lower case.
    :param obj: suds object
    :return: dict object
    """
    import datetime

    if not hasattr(obj, '__keylist__'):
        if json_serialize and isinstance(obj, (datetime.datetime, datetime.time, datetime.date)):
            return obj.isoformat()
        else:
            return obj
    data = {}
    fields = obj.__keylist__
    for field in fields:
        val = getattr(obj, field)
        if key_to_lower:
            field = field.lower()
        if isinstance(val, list):
            data[field] = []
            for item in val:
                data[field].append(sobject_to_dict(item, json_serialize=json_serialize))
        else:
            data[field] = sobject_to_dict(val, json_serialize=json_serialize)
    return data


def sobject_to_json(obj, key_to_lower=False):
    """
    Converts a suds object to json.
    :param obj: suds object
    :param key_to_lower: If set, changes index key name to lower case.
    :return: json object
    """
    data = sobject_to_dict(obj, key_to_lower=key_to_lower, json_serialize=True)
    return json.dumps(data)

firmindex = 0
dobject = []
for response in d.keys():
    firmlist = []
    print ("Started converting data of firm "+str(firmindex))
    for r in d[response].keys():
        test = sobject_to_json(d[response][r])
        jsonobject = json.loads(test)
        # Debugging: Show Instrument (= DataStream query)
        # print (jsonobject['Instrument'])
        # Debugging: Print complete JSON object
        # print (jsonobject)
        try:
            for i in jsonobject['Fields']['Field']:
                itemlist = []
                # Debugging: Show single items included in response
                # print ("Loop over Field-List-Items that are dictionaries")
                # Debugging: Show item name
                # print (i['Name'])
                itemlist.append(i['Name'])
                try:
                    # Debugging: Check if content is Array and show contents
                    # print ("Is Dictionary.")
                    # print (i['ArrayValue'])
                    for value in i['ArrayValue']['anyType']:
                        # Debugging: Show values in array
                        # print (value)
                        itemlist.append(value)
                except KeyError:
                    # Debugging: Content must be single value, show value
                    # print ("Is Value.")
                    # print (i['Value'])
                    itemlist.append(i['Value'])
                firmlist.append(itemlist)
        except TypeError:
            pass
    dobject.append(firmlist)
    print ("Finished converting data of firm "+str(firmindex))
    firmindex = firmindex + 1
print ("Conversion completed.")
# Debugging: Show complete converted data object
# print ("Complete Data Object:")
# print (dobject)

Started converting data of firm 0
Finished converting data of firm 0
Started converting data of firm 1
Finished converting data of firm 1
Started converting data of firm 2
Finished converting data of firm 2
Started converting data of firm 3
Finished converting data of firm 3
Started converting data of firm 4
Finished converting data of firm 4
Conversion completed.


## Export to CSV

For each firm, `dobject` is converted to a `pandas` dataframe. Data is reshaped to conform to the *long format*. All originating dataframes are merged.

In most cases, not all items are available. To deal with the issue of missing data values (and the resulting `key indexing error` for the lists), numerous `try` and `except` specification are included.

Results are written to the semicolon-separated CSV file called `ds_output.csv`. 

Excerpt from the resulting file (first two items):

```
;ccy;date;dispname;frequency;insterror;symbol;wc01149;wc01150;
0;;2005-12-31T00:00:00;CENTROTEC SUSTAIN;Y;"FAILED: $$""ER"", 4540, NO DATA VALUES FOUND";688137;932.0;;17763.0;
```

In [6]:
import pandas as pd

df = pd.DataFrame.from_records(dobject[0])
df = df.transpose()
varnames = df.iloc[0]
j = 0
for i in range(0, len(varnames)):
    try:
        # Debugging: Show all variables in first response data set
        # print (varnames[i])
        df.rename(columns={j: varnames[i].lower()}, inplace=True)
    except KeyError:
        pass
    j += 1
df.drop(0, inplace=True)
df = df.loc[:,~df.columns.duplicated()]
try:
    df['dispname'].fillna(method='ffill', inplace=True)
except KeyError:
    pass
try:
    df['frequency'].fillna(method='ffill', inplace=True)
except KeyError:
    pass
try:
    df['insterror'].fillna(method='ffill', inplace=True)
except KeyError:
    pass
try:
    df['symbol'].fillna(method='ffill', inplace=True)
except KeyError:
    pass
# Debugging: Show first data set
# print("Start Data set:")
# print(df)

k = 1
for i in range(1,len(dobject)):
    df2 = pd.DataFrame.from_records(dobject[i])
    df2 = df2.transpose()
    varnames = df2.iloc[0]
    j = 0
    for i in range(0, len(varnames)):
        try:
            # Debugging: Show variables in using data set
            # print (varnames[i])
            df2.rename(columns={j: varnames[i].lower()}, inplace=True)
        except KeyError:
            pass
        j += 1
    df2.drop(0, inplace=True)
    df2 = df2.loc[:,~df2.columns.duplicated()]
    try:
        df2['dispname'].fillna(method='ffill', inplace=True)
    except KeyError:
        pass
    try:
        df2['frequency'].fillna(method='ffill', inplace=True)
    except KeyError:
        pass
    try:
        df2['insterror'].fillna(method='ffill', inplace=True)
    except KeyError:
        pass
    try:
        df2['symbol'].fillna(method='ffill', inplace=True)
    except KeyError:
        pass
    # Debugging: Show using data set
    # print("Using data set:")
    # print(df2)
    df = pd.concat([df, df2], ignore_index=True)
    print ("Merge "+str(k)+" of "+str(len(dobject)-1)+" completed.")
    # Debugging: Show merged data set
    # print("Merged data set:")
    # print(df)
    k += 1

# write result to csv file
df.to_csv('ds_output.csv', sep=';', encoding='utf-8')
print ("CSV Export completed.")

Merge 1 of 4 completed.
Merge 2 of 4 completed.
Merge 3 of 4 completed.
Merge 4 of 4 completed.
CSV Export completed.
